# TypeScript Counter 問題の完全解析

## 1. 問題の分析

### 競技プログラミング視点での分析
- **実行速度最優先**: クロージャーを利用した O(1) の定数時間アクセス
- **メモリ使用量**: 単一の数値変数のみを保持 (O(1) 空間)
- **最適化ポイント**: プリミティブ型の直接操作、不要なオブジェクト生成の回避

### 業務開発視点での分析
- **型安全性**: 引数と戻り値の厳密な型定義が必要
- **保守性**: クロージャーの仕組みを明確にドキュメント化
- **エラーハンドリング**: 制約条件 (-1000 ≤ n ≤ 1000) のバリデーション
- **予測可能性**: Pure function ではなく状態を持つが、副作用は限定的

### TypeScript特有の考慮点
- **型推論の活用**: 戻り値の型を明示的に定義
- **ジェネリクス**: この問題では不要（number型固定）
- **型ガード**: 入力値の範囲検証
- **クロージャー型定義**: 関数型の明示的な型注釈

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---------|----------|----------|------------|---------|--------|------|
| クロージャー + 後置インクリメント | O(1) | O(1) | 低 | 高 | 高 | 最もシンプル、LeetCodeの想定解 |
| クロージャー + 前置インクリメント | O(1) | O(1) | 低 | 高 | 中 | 初回呼び出し時の処理が複雑化 |
| オブジェクト指向 (class) | O(1) | O(1) | 中 | 高 | 中 | 過剰設計、LeetCode形式に不適 |
| Generator関数 | O(1) | O(1) | 中 | 中 | 低 | 呼び出し形式が異なる |

## 3. 選択したアルゴリズムと理由

### 選択したアプローチ
**クロージャー + 後置インクリメント (n++) 方式**

### 理由

#### 計算量的な優位性
- 時間計算量: O(1) - 各呼び出しで定数時間
- 空間計算量: O(1) - 単一の数値変数のみ
- インクリメント演算は CPU レベルで最適化済み

#### TypeScript環境での型安全性
- 引数 `n: number` の厳密な型定義
- 戻り値の関数型を明示: `() => number`
- コンパイル時に型不一致を検出可能

#### 保守性・可読性の観点
- クロージャーパターンは JavaScript/TypeScript の標準的なイディオム
- 後置インクリメント (`n++`) により「現在値を返してから増加」が自明
- コード量が最小で理解しやすい

### TypeScript特有の最適化ポイント
- **厳密な型定義**: strict mode でのコンパイル時検証
- **readonly 不要**: クロージャー内の変数は外部から直接アクセス不可
- **型推論の活用**: 内部変数 `n` の型は自動推論される

## 4. 実装コード

```typescript
/**
 * カウンター関数を生成する
 * 初回呼び出し時は初期値nを返し、以降呼び出すたびに1ずつ増加した値を返す
 * 
 * @param n - カウンターの初期値 (-1000 <= n <= 1000)
 * @returns 呼び出すたびにインクリメントされる値を返す関数
 * @throws {RangeError} nが制約範囲外の場合
 * @complexity Time: O(1), Space: O(1)
 * 
 * @example
 * const counter = createCounter(10);
 * counter(); // 10
 * counter(); // 11
 * counter(); // 12
 */
function createCounter(n: number): () => number {
    // 入力検証: 制約条件のチェック
    if (n < -1000 || n > 1000) {
        throw new RangeError('Initial value must be between -1000 and 1000');
    }
    
    // 型ガード: number型の確認
    if (typeof n !== 'number' || !Number.isFinite(n)) {
        throw new TypeError('Initial value must be a finite number');
    }
    
    /**
     * クロージャーによるカウンター実装
     * 外部スコープの変数nを保持し、呼び出すたびにインクリメント
     * 
     * @returns 現在のカウント値（呼び出し後に内部状態が+1される）
     */
    return function(): number {
        // 後置インクリメント: 現在値を返した後にnを+1
        // この演算子により「返却」→「増加」の順序が保証される
        return n++;
    };
}

// LeetCode形式のエクスポート（互換性のため）
const createCounterLeetCode = createCounter;

/**
 * 使用例とテストケース
 */
// Example 1
const counter1 = createCounter(10);
console.assert(counter1() === 10, 'First call should return 10');
console.assert(counter1() === 11, 'Second call should return 11');
console.assert(counter1() === 12, 'Third call should return 12');

// Example 2
const counter2 = createCounter(-2);
console.assert(counter2() === -2, 'First call should return -2');
console.assert(counter2() === -1, 'Second call should return -1');
console.assert(counter2() === 0, 'Third call should return 0');
console.assert(counter2() === 1, 'Fourth call should return 1');
console.assert(counter2() === 2, 'Fifth call should return 2');

// エッジケース
const counterMin = createCounter(-1000);
console.assert(counterMin() === -1000, 'Minimum value test');

const counterMax = createCounter(1000);
console.assert(counterMax() === 1000, 'Maximum value test');
```

## LeetCode提出用フォーマット

```typescript
// Analyze Complexity
// Runtime 43 ms
// Beats 57.94%
// Memory 53.82 MB
// Beats 89.21%

/**
 * @param {number} n
 * @return {Function} counter
 */
function createCounter(n: number): () => number {
    return function(): number {
        return n++;
    };
}

/** 
 * const counter = createCounter(10)
 * counter() // 10
 * counter() // 11
 * counter() // 12
 */
```

## TypeScript固有の最適化観点

### 型安全性の活用

1. **コンパイル時エラー防止**
   - `n: number` により文字列や他の型の混入を防止
   - `() => number` により戻り値の型不一致を検出
   - strict モードで `null`/`undefined` の混入を防止

2. **実行時型検証**
   ```typescript
   if (typeof n !== 'number' || !Number.isFinite(n)) {
       throw new TypeError('Invalid input type');
   }
   ```

3. **型推論の活用**
   - クロージャー内の `n` は自動的に `number` 型と推論される
   - 明示的な型注釈が不要で冗長性を削減

### パフォーマンス最適化

1. **プリミティブ型の直接操作**
   - オブジェクト生成のオーバーヘッドなし
   - 後置インクリメント演算子は CPU レベルで最適化

2. **クロージャーの効率的利用**
   - 変数 `n` はヒープではなくクロージャースコープに保持
   - ガベージコレクションの対象が最小化

3. **不要な処理の排除**
   - 条件分岐なし
   - 配列やオブジェクトの生成なし

### 保守性と開発効率

- **IntelliSense サポート**: 関数の型情報が IDE で自動表示
- **リファクタリング安全性**: 型定義により変更影響範囲を把握
- **自己文書化**: JSDoc コメントと型定義でドキュメント不要
- **テスト容易性**: Pure な入出力で単体テストが簡単

この実装は、LeetCode の制約条件を満たしつつ、TypeScript の型安全性を最大限活用した最適解となっています。